In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
//import $ivy.`sh.almond::almond-spark:0.3
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`
import $ivy.`org.vegas-viz::vegas:0.3.10`

import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.apache.log4j.{Level, Logger}
import scala.collection.JavaConverters._
import java.io._

Logger.getLogger("org").setLevel(Level.OFF)

val spark = AmmoniteSparkSession.builder()
    .config("spark.serializer",classOf[KryoSerializer].getName)
    .master("local[*]").appName("Areal")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId


Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here
//import $ivy.`sh.almond::almond-spark:0.3

import $ivy.$                                

import $ivy.$                              

import $ivy.$                            


import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.apache.spark.sql._

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.apache.log4j.{Level, Logger}

import scala.collection.JavaConverters._

import java.io._


spark: SparkSession = org.apache.spark.sql.SparkSession@4a98fa02
import spark.implicits._

appID: String = "local-1555960667337"

In [33]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Log(timestamp: String, duration: Int, appID: String, nodes: String, id: Int, worker: String, stage: String
               , rddBlocks: Int, tasksCompleted: Int, nodeTime: Double , tasksLoad: Double)
val data = spark.read.option("header", "false").option("delimiter", "|")
    .csv("/home/acald013/Research/Validation/MultinodeScaleup/monitor.log")
    .map{ l => 
        val timestamp = l.getString(0)
        val duration = l.getString(1).toInt
        val appID    = l.getString(2)
        val nodes    = l.getString(3).toInt
        val id       = l.getString(4).toInt
        val worker   = l.getString(5)
        val stage    = l.getString(6)
        val rdds     = l.getString(7).toInt
        val comp     = l.getString(8).toInt
        val dura     = l.getString(9).toDouble
        val load     = l.getString(10).toDouble
        
        (timestamp, duration, appID, nodes, id, worker, stage, rdds, comp, dura, load)
    }.toDF("timestamp", "duration", "appID", "nodes", "id", "worker", "stage", "rddBlocks", "tasksCompleted", "nodeTime", "tasksLoad").as[Log]

defined class Log
data: Dataset[Log] = [timestamp: string, duration: int ... 9 more fields]

In [34]:
data.persist(StorageLevel.MEMORY_ONLY)
data.show(truncate=false)

+-----------------------+--------+-----------------------+-----+---+--------------------+-----------------------+---------+--------------+--------+---------+
|timestamp              |duration|appID                  |nodes|id |worker              |stage                  |rddBlocks|tasksCompleted|nodeTime|tasksLoad|
+-----------------------+--------+-----------------------+-----+---+--------------------+-----------------------+---------+--------------+--------+---------+
|2019-04-20 11:52:54,707|33      |app-20190420115223-0000|3    |2  |169.235.27.135:33775|flatMap at MF.scala:153|1037     |1354          |20.96   |519.73   |
|2019-04-20 11:52:54,707|33      |app-20190420115223-0000|3    |1  |169.235.27.137:45856|flatMap at MF.scala:153|1056     |1358          |21.63   |556.5    |
|2019-04-20 11:52:54,707|33      |app-20190420115223-0000|3    |0  |169.235.27.134:40730|flatMap at MF.scala:153|1139     |1513          |24.74   |658.93   |
|2019-04-20 11:52:54,967|33      |app-20190420115223

res33_0: Dataset[Log] = [timestamp: string, duration: int ... 9 more fields]

In [22]:
val apps = data.select("appID").distinct().orderBy("appID").cache()
apps.show(truncate = false)

+-----------------------+
|appID                  |
+-----------------------+
|app-20190420115223-0000|
|app-20190420120928-0001|
|app-20190420122227-0002|
|app-20190420123748-0003|
|app-20190420125022-0004|
|app-20190420130734-0005|
|app-20190420132031-0006|
|app-20190420133750-0007|
|app-20190420135120-0008|
|app-20190420140717-0009|
+-----------------------+



apps: Dataset[Row] = [appID: string]

In [26]:
def extractAppRates(appID: String, data: Dataset[Log]): DataFrame = {
    val node_info = data.filter(_.appID == appID).select($"duration", $"id", $"worker", $"rddBlocks", $"tasksCompleted", $"tasksLoad")
        .groupBy($"duration", $"id", $"worker")
        .agg(mean($"rddBlocks").as("rdds"), mean($"tasksCompleted").as("task"), mean($"tasksLoad").as("load"))
        .withColumn("rddsRate", $"rdds" / $"duration")
        .withColumn("taskRate", $"task" / $"duration")
        .withColumn("loadRate", $"load" / $"duration")
        .select($"duration", $"worker", $"rddsRate", $"taskRate", $"loadRate")
        .orderBy($"duration", $"worker")
    val pw = new PrintWriter(s"/tmp/${appID}.tsv")
    pw.write(node_info.map{ m => 
        s"${m.getInt(0)}\t${m.getString(1)}\t${m.getDouble(2)}\t${m.getDouble(3)}\t${m.getDouble(4)}\n"
    }.collect().mkString(""))
    pw.close()
    
    node_info
}

defined function extractAppRates

In [19]:
val appID = "app-20190420123748-0003"
val tm = extractAppRates(appID, data)
tm.show(truncate=false)

+--------+--------------------+--------------------+-------------------+--------------------+
|duration|worker              |rddsRate            |taskRate           |loadRate            |
+--------+--------------------+--------------------+-------------------+--------------------+
|3       |169.235.27.134:40431|0.0                 |0.0                |0.0                 |
|4       |169.235.27.134:40431|0.0                 |0.0                |0.0                 |
|5       |169.235.27.134:40431|0.0                 |0.03333333333333333|0.0                 |
|6       |169.235.27.134:40431|0.0                 |0.25396825396825395|0.050873015873015875|
|7       |169.235.27.134:40431|0.006211180124223602|2.3540372670807455 |1.4332919254658378  |
|8       |169.235.27.134:40431|1.5773809523809523  |4.738095238095238  |2.8251190476190464  |
|9       |169.235.27.134:40431|3.587301587301587   |6.412698412698413  |4.258201058201059   |
|10      |169.235.27.134:40431|4.747368421052632   |9.042105

appID: String = "app-20190420123748-0003"
tm: DataFrame = [duration: int, worker: string ... 3 more fields]

In [25]:
for(app <- apps.collect()){
    extractAppRates(app.getString(0), data)
}

In [29]:
def extractAppMetrics(appID: String, data: Dataset[Log]): DataFrame = {
    val node_info = data.filter(f => f.appID == appID && f.worker.contains(":"))
        .select($"duration", $"worker", $"rddBlocks", $"tasksCompleted", $"tasksLoad")
        .groupBy($"duration", $"worker")
        .agg(max($"rddBlocks").as("rdds"), max($"tasksCompleted").as("task"), max($"tasksLoad").as("load"))
        .select($"duration", $"worker", $"rdds", $"task", $"load")
        .orderBy($"duration", $"worker")
    val fileID = appID.split("-")(2)
    val pw = new PrintWriter(s"/tmp/M${fileID}.tsv")
    pw.write(node_info.map{ m => 
        s"${m.getInt(0)}\t${m.getString(1)}\t${m.getInt(2)}\t${m.getInt(3)}\t${m.getDouble(4)}\n"
    }.collect().mkString(""))
    pw.close()
    
    node_info
}

defined function extractAppMetrics

In [31]:
for(app <- apps.collect()){
    extractAppMetrics(app.getString(0), data)
}

In [51]:
val starttimes = spark.read.option("header", "false").option("delimiter", "\t").csv("/home/acald013/Research/Validation/starttimes.csv")
    .toDF("metric", "appID","epoch").withColumn("ts0", unix_timestamp($"epoch", "yyyy-MM-dd HH:mm:ss")).cache().cache()
starttimes.show()

+------+--------------------+--------------------+----------+
|metric|               appID|               epoch|       ts0|
+------+--------------------+--------------------+----------+
| M0000|app-2019042011522...|2019-04-20 11:52:...|1555786341|
| M0001|app-2019042012092...|2019-04-20 12:09:...|1555787367|
| M0002|app-2019042012222...|2019-04-20 12:22:...|1555788145|
| M0003|app-2019042012374...|2019-04-20 12:37:...|1555789066|
| M0004|app-2019042012502...|2019-04-20 12:50:...|1555789820|
| M0005|app-2019042013073...|2019-04-20 13:07:...|1555790852|
| M0006|app-2019042013203...|2019-04-20 13:20:...|1555791630|
| M0007|app-2019042013375...|2019-04-20 13:37:...|1555792669|
| M0008|app-2019042013512...|2019-04-20 13:51:...|1555793479|
| M0009|app-2019042014071...|2019-04-20 14:07:...|1555794435|
+------+--------------------+--------------------+----------+



starttimes: Dataset[Row] = [metric: string, appID: string ... 2 more fields]

In [52]:
val nohup = spark.read.option("header", "false").option("delimiter", "|").csv("/home/acald013/Research/Validation/MultinodeScaleup/stages.psv")
    .toDF("epoch", "appID", "stage", "time", "load", "timestamp").withColumn("ts1", unix_timestamp($"epoch", "yyyy-MM-dd HH:mm:ss")).cache()
nohup.show(truncate = false)

+-----------------------+-----------------------+--------------------------------------------------+------+------+---------+----------+
|epoch                  |appID                  |stage                                             |time  |load  |timestamp|ts1       |
+-----------------------+-----------------------+--------------------------------------------------+------+------+---------+----------+
|2019-04-20 11:53:44,394|app-20190420115223-0000|1.Maximal disks found                             | 72.34| 39886|0        |1555786424|
|2019-04-20 11:54:39,949|app-20190420115223-0000|1.Maximal disks found                             | 55.33| 40330|1        |1555786479|
|2019-04-20 11:54:48,052|app-20190420115223-0000|2.Join done                                       |  8.10|128740|1        |1555786488|
|2019-04-20 11:54:49,418|app-20190420115223-0000|3.Flocks to report                                |  1.37|     0|1        |1555786489|
|2019-04-20 11:54:49,478|app-20190420115223-0000

nohup: Dataset[Row] = [epoch: string, appID: string ... 5 more fields]

In [55]:
val joined = nohup.join(starttimes, "appID").select("appID", "timestamp", "stage", "ts1", "ts0").cache()
joined.show(truncate = false)

+-----------------------+---------+--------------------------------------------------+----------+----------+
|appID                  |timestamp|stage                                             |ts1       |ts0       |
+-----------------------+---------+--------------------------------------------------+----------+----------+
|app-20190420115223-0000|0        |1.Maximal disks found                             |1555786424|1555786341|
|app-20190420115223-0000|1        |1.Maximal disks found                             |1555786479|1555786341|
|app-20190420115223-0000|1        |2.Join done                                       |1555786488|1555786341|
|app-20190420115223-0000|1        |3.Flocks to report                                |1555786489|1555786341|
|app-20190420115223-0000|1        |4.Flocks prunned by expansion                     |1555786489|1555786341|
|app-20190420115223-0000|1        |5.Flocks reported                                 |1555786489|1555786341|
|app-20190420115223

joined: Dataset[Row] = [appID: string, timestamp: string ... 3 more fields]

In [57]:
val stages = joined.map{ j => 
    val appID = j.getString(0)
    val time = j.getLong(3) - j.getLong(4)
    val stage = s"${j.getString(1)}.${j.getString(2)}"
    (appID, time, stage)
}
stages.show(truncate = false)

+-----------------------+---+----------------------------------------------------+
|_1                     |_2 |_3                                                  |
+-----------------------+---+----------------------------------------------------+
|app-20190420115223-0000|83 |0.1.Maximal disks found                             |
|app-20190420115223-0000|138|1.1.Maximal disks found                             |
|app-20190420115223-0000|147|1.2.Join done                                       |
|app-20190420115223-0000|148|1.3.Flocks to report                                |
|app-20190420115223-0000|148|1.4.Flocks prunned by expansion                     |
|app-20190420115223-0000|148|1.5.Flocks reported                                 |
|app-20190420115223-0000|157|1.6.Candidates indexed                              |
|app-20190420115223-0000|232|2.1.Maximal disks found                             |
|app-20190420115223-0000|282|2.2.Join done                                       |
|app

stages: Dataset[(String, Long, String)] = [_1: string, _2: bigint ... 1 more field]

In [58]:
val pw = new PrintWriter(s"/tmp/stages.tsv")
pw.write(stages.map{ m => 
    s"${m._1}\t${m._2}\t${m._3}\n"
}.collect().mkString(""))
pw.close()

pw: PrintWriter = java.io.PrintWriter@3412b1ef